In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
DATA_DIR = "data/icra2025-v0"


In [3]:
# remove all "wav" files from the data directory
import os
import glob
# glob cannot find files with a leading dot
files = glob.glob(os.path.join(DATA_DIR, "**", "*.wav"), recursive=True)

for f in files:
    os.remove(f)

In [4]:
import os

In [5]:
# find all csv files in the data directory
categories = os.listdir(DATA_DIR)
categories = [c for c in categories if os.path.isdir(os.path.join(DATA_DIR, c))]
print(categories)


import pandas as pd

# create df with 3 columns: filepath, sensor_type, category
entries = []

# for each category, find all csv files
for category in categories:
    print(category)
    csv_files = os.listdir(os.path.join(DATA_DIR, category))
    csv_files = [f for f in csv_files if f.endswith(".csv")]
    print(csv_files)

    for file in csv_files:
        sensor_type = file.split("_")[0]
        filepath = os.path.join(DATA_DIR, category, file)
        # add to df
        entries.append({"filepath": filepath, "sensor_type": sensor_type, "category": category})    

df = pd.DataFrame(entries)

print(len(df))

['3_M6x14', 'empty', 'empty_disturbed']
3_M6x14
['laser_2025-02-18[15].csv', 'mic_2025-02-18[20].csv', 'mic_2025-02-18[11].csv', 'laser_2025-02-18[9].csv', 'laser_2025-02-18[17].csv', 'laser_2025-02-18[14].csv', 'mic_2025-02-18[17].csv', 'mic_2025-02-18[6].csv', 'laser_2025-02-18[16].csv', 'mic_2025-02-18[1].csv', 'laser_2025-02-18[18].csv', 'laser_2025-02-18[8].csv', 'mic_2025-02-18[2].csv', 'laser_2025-02-18[20].csv', 'mic_2025-02-18[5].csv', 'mic_2025-02-18[13].csv', 'laser_2025-02-18[12].csv', 'laser_2025-02-18[5].csv', 'laser_2025-02-18[19].csv', 'mic_2025-02-18[9].csv', 'laser_2025-02-18[3].csv', 'laser_2025-02-18[1].csv', 'mic_2025-02-18[7].csv', 'mic_2025-02-18[15].csv', 'mic_2025-02-18[10].csv', 'mic_2025-02-18[19].csv', 'laser_2025-02-18[6].csv', 'mic_2025-02-18[14].csv', 'mic_2025-02-18[16].csv', 'mic_2025-02-18[4].csv', 'laser_2025-02-18[4].csv', 'mic_2025-02-18[12].csv', 'laser_2025-02-18[10].csv', 'laser_2025-02-18[2].csv', 'laser_2025-02-18[13].csv', 'mic_2025-02-18[18].

In [6]:
# drop category 'empty_disturbed'
df = df[df["category"] != "empty_disturbed"]
print(len(df))


80


In [7]:
def plot_waveform(waveform, sample_rate):

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("waveform")

In [8]:
# for each entry, read the csv file save wav file
from converter import SpectrogramCalculator
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

for i, row in tqdm(df.iterrows(), total=len(df)):
 
    filepath = row["filepath"]
    par_dir = os.path.dirname(filepath)
    filename = os.path.basename(filepath)
    filename = filename.replace(".csv", "")


    calc = SpectrogramCalculator(par_dir, filename)


    calc.encode_as_wav()

    # plot waveform
    # plot_waveform(torch.tensor(calc.data).unsqueeze(0), int(calc.fs))
    # plt.show()
    del calc
    


100%|██████████| 80/80 [00:34<00:00,  2.35it/s]


In [9]:
laser_df = df[df["sensor_type"] == "laser"]
mic_df = df[df["sensor_type"] == "mic"]

In [10]:
import random
random.seed(42)

# create train and test splits 
val_ratio = 0.2
train_laser_indices = random.sample(range(len(laser_df)), int(len(laser_df) * (1 - val_ratio)))
val_laser_indices = [i for i in range(len(laser_df)) if i not in train_laser_indices]

train_mic_indices = random.sample(range(len(mic_df)), int(len(mic_df) * (1 - val_ratio)))
val_mic_indices = [i for i in range(len(mic_df)) if i not in train_mic_indices]

train_laser_df = laser_df.iloc[train_laser_indices]
val_laser_df = laser_df.iloc[val_laser_indices]

train_mic_df = mic_df.iloc[train_mic_indices]
val_mic_df = mic_df.iloc[val_mic_indices]


In [11]:
# create train dataset.